# Time convergence

strong form of the initial boundary value problem
$$
\begin{align*}
&\text{Find $u(\textbf{x}, t): \Omega\times[0,\infty) \to \mathbb{R}$ such that } \\
&\begin{cases}
\frac{\partial u}{\partial t} = \mathcal{L}(u) & \forall(\textbf{x}, t)\in\Omega\times[0,\infty) \\
u(\textbf{x}, t=0)=u_0 & \forall\textbf{x}\in\Omega\\
u=u_{\text{D}} & \forall \textbf{x}\in\partial\Omega 
\end{cases}~.
\end{align*}
$$

Given a fixed timestep $\Delta t$ and mesh cell size $h$, the error norm at time $t$ is expected to scale as

$$||e(t)||\sim C_{\Delta t}\Delta t^{\,r_{\Delta t}} + C_hh^{r_h}$$

with $C_{\Delta t}, C_h, r_\Delta, r_h$ themselves being time-dependent. We also consider the maximum-in-time error norm

$$\max_{t\geq 0}||e(t)|| = \max_{0\leq n \leq n_{\text{stop}}}||e(n\Delta t)||$$

## Example: convergence of the diffusion equation on a unit square

$$
\mathbb{S}=
\begin{cases}
\Omega = [0, 1] \times [0, 1] \\
\mathcal{L}=\nabla^2 \\
u_{\text{e}}(x,y,t)=e^{-\nu t}\cos(2\pi x)\cos(2\pi y) \\
\end{cases}
$$

In [ ]:
from typing import Callable
import numpy as np
from ufl.core.expr import Expr
from ufl import SpatialCoordinate, cos
from dolfinx.fem import FunctionSpace

from lucifex.mesh import rectangle_mesh, mesh_boundary
from lucifex.fem import Function, Constant
from lucifex.fdm import FunctionSeries, ConstantSeries, ExprSeries, FiniteDifference, BE
from lucifex.fdm.ufl_operators import exp
from lucifex.solver import ibvp, integration, BoundaryConditions
from lucifex.viz import plot_colormap, plot_line
from lucifex.utils import (
    L_norm, set_finite_element_function, 
    interpolate_finite_element_function, nested_dict
)
from lucifex.sim import run, Simulation
from lucifex.pde.foundations import diffusion



def run_simulations(
    Nx: int,
    u_exact: Callable[[SpatialCoordinate, ConstantSeries], ExprSeries],
    cell: str,
    family: str,
    degree: int,
    dt: float,
    n_stop: int,
    fdm: FiniteDifference,
    store: int,
) -> tuple[float, Simulation]:
    Lx = 1.0
    h = Lx / Nx
    mesh = rectangle_mesh(Lx, Lx, Nx, Nx, cell)
    boundary = mesh_boundary(
        mesh, 
        {
            "left": lambda x: x[0],
            "right": lambda x: x[0] - Lx,
            "lower": lambda x: x[1],
            "upper": lambda x: x[1] - Lx,
        },
        complete=True,
    )
    t = ConstantSeries(mesh, name='t', ics=0.0)
    dt = Constant(mesh, dt, name='dt')
    order = max(fdm.order, 1)

    x = SpatialCoordinate(mesh)
    ue = u_exact(x, t)
    ue.name = 'ue'
    u = FunctionSeries(
        (mesh, family, degree), 
        'u', 
        order, 
        store, 
        ics=lambda x: u_exact(x, 0.0),
    )

    bcs = BoundaryConditions(
        ('dirichlet', boundary.union, ue[1])
    )
    u_solver = ibvp(diffusion, bcs=bcs)(u, dt, 1, fdm)

    norm = ConstantSeries(mesh, 'norm', order, store=store)
    norm_solver = integration(norm, L_norm, 'dx')(ue[0] - u[0], 2.0)

    simulation = Simulation(
        [u_solver, norm_solver],
        t, 
        dt,
        [ue],
    )
    run(simulation, n_stop=n_stop)

    return h, simulation


nu = 1.0
u_exact = lambda x, t: exp(-nu * t) * cos(2 * np.pi * x[0]) * cos(2 * np.pi * x[1])

Nx = (10, 50, 100, 200)
Dt = (5e-2, 1e-2, 5e-3, 1e-4)
n_stop = 10

solutions = nested_dict()

for nx in Nx:
    for dt in Dt:
        h, sim = run_simulations(nx, u_exact, 'triangle', 'P', 1, dt, n_stop, BE, 1)


AssertionError: 